In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! GPU is ready to use.")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Us")


CUDA is available! GPU is ready to use.
GPU Name: NVIDIA GeForce GTX 1070


In [5]:
import pandas as pd
import json

In [2]:
# load model
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

c:\Users\tom_r\anaconda3\envs\Generative_AI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tom_r\anaconda3\envs\Generative_AI\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tom_r\.cache\huggingface\hub\models--distilbert--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order 

In [3]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [18]:
with open(r"C:\Users\tom_r\Downloads\triviaqa-rc\qa\wikipedia-train.json", 'r') as f:
    data = json.load(f)

print(data.keys()) 
print(data['Data'][0]) # look at some examples
print(data['Data'][1]) 

dict_keys(['Data', 'Domain', 'Split', 'VerifiedEval', 'Version'])
{'Answer': {'Aliases': ['Park Grove (1895)', 'York UA', 'Yorkish', 'UN/LOCODE:GBYRK', 'York, UK', 'Eoforwic', 'Park Grove School', 'York Ham', 'The weather in York', 'City of York', 'York, England', 'York, Yorkshire', 'York ham', 'County Borough of York', 'YORK', 'Eoferwic', 'Park Grove Primary School', 'York, North Yorkshire', 'Yoisk', 'York', 'York (England)'], 'MatchedWikiEntityName': 'York', 'NormalizedAliases': ['york yorkshire', 'eoferwic', 'park grove primary school', 'park grove school', 'weather in york', 'park grove 1895', 'eoforwic', 'county borough of york', 'york uk', 'un locode gbyrk', 'city of york', 'york england', 'york ua', 'york ham', 'york', 'yorkish', 'yoisk', 'york north yorkshire'], 'NormalizedMatchedWikiEntityName': 'york', 'NormalizedValue': 'york', 'Type': 'WikipediaEntity', 'Value': 'York'}, 'EntityPages': [{'DocSource': 'TagMe', 'Filename': 'England.txt', 'Title': 'England'}, {'DocSource': 'Ta

In [44]:
print(len(data['Data']))

61888


In [49]:
# extracting relevant fields

def load_evidence(filename):
    with open(fr'C:\Users\tom_r\Downloads\triviaqa-rc\evidence\wikipedia\{filename}', 'r') as f:
        return f.read()

questions = []
answers = []
evidence = []

for item in data['Data']:

    try:
        
        evidence_docs = []
        for page in item['EntityPages']:
            evidence_docs.append(load_evidence(page['Filename']))
        evidence.append(evidence_docs)
        
        questions.append(item['Question'])
        answers.append(item['Answer'])

        
    except:
        pass

In [52]:
print(len(questions))
print(len(answers))
print(len(evidence))

print(questions[1])
print(answers[1])
print(evidence[1])

print(questions[20000])
print(answers[20000])
print(evidence[20000])

61599
61599
61599
From which country did Angola achieve independence in 1975?
{'Aliases': ['Portogało', 'Republic of Portugal', 'PORTUGAL', 'Portekiz', 'Portugallu', 'O Papagaio', 'ISO 3166-1:PT', 'Portunga', 'Phu-to-ga', 'Potigal', 'Portûnga', 'Portugul', 'An Phortaingéil', 'Portugāle', 'Portugale', 'Portingale', 'Potiti', 'Portugali', 'Portugall', 'Portekîz', 'Bo Dao Nha', 'Portuguese Republic', 'Portogallo', 'Portugaul', 'Portogalo', 'Portyngal', 'Yn Phortiugal', 'Portugalio', 'Portugál', 'Portugual', 'Portuga', 'Portgual', 'Portugalsko', 'Portugaleje', 'Phû-tô-gâ', 'Portugalujo', 'Portugalija', 'Pertual', 'Pòtigal', 'Portugal', 'Bồ Đào Nha', 'Portugalska', 'República Portuguesa', 'Portiwgal', 'Portugalėjė', 'Portúgal', 'Portegal', 'An Phortaingeil', 'Republica Portuguesa'], 'MatchedWikiEntityName': 'Portugal', 'NormalizedAliases': ['portugul', 'portugallu', 'portugalska', 'pòtigal', 'portugaul', 'portugalujo', 'portuguese republic', 'iso 3166 1 pt', 'republic of portugal', 'portuga